## Definint funcions:

In [75]:
def ImportData(dbPath, bSize, nTrain, nVal):
    params = [trn.ASENeighborList(cutoff=5.),
              trn.RemoveOffsets(QM9.U0, remove_mean=True, 
              remove_atomrefs=True),
              trn.CastTo32()]
    
    qm9data = QM9(dbPath, 
        batch_size = bSize,
        num_train = nTrain, 
        num_val = nVal,
        transforms = params,
        property_units={QM9.U0: 'eV'},
        num_workers=1,
        split_file=os.path.join(qm9tut, "split.npz"),
        pin_memory=True, # set to false, when not using a GPU
        load_properties=[QM9.U0], #only load U0 property
    )
    qm9data.prepare_data()
    qm9data.setup()
    return qm9data

In [76]:
def settingModel(cutoff, n_atom_basis):
    pairwise_distance = spk.atomistic.PairwiseDistances() # calculates pairwise distances between atoms
    radial_basis = spk.nn.GaussianRBF(n_rbf=20, cutoff=cutoff)
    schnet = spk.representation.SchNet(
        n_atom_basis=n_atom_basis, n_interactions=3,
        radial_basis=radial_basis,
        cutoff_fn=spk.nn.CosineCutoff(cutoff)
    )
    pred_U0 = spk.atomistic.Atomwise(n_in=n_atom_basis, output_key=QM9.U0)

    nnpot = spk.model.NeuralNetworkPotential(
        representation=schnet,
        input_modules=[pairwise_distance],
        output_modules=[pred_U0],
        postprocessors=[trn.CastTo64(), trn.AddOffsets(QM9.U0, add_mean=True, add_atomrefs=True)]
    )
    return nnpot

In [77]:
def settingOutput():
    output_U0 = spk.task.ModelOutput(
        name=QM9.U0,
        loss_fn=torch.nn.MSELoss(),
        loss_weight=1.,
        metrics={
            "MAE": torchmetrics.MeanAbsoluteError()
        }
    )
    return output_U0

In [78]:
def definingTasck():
    task = spk.task.AtomisticTask(
        model=nnpot,
        outputs=[output_U0],
        optimizer_cls=torch.optim.AdamW,
        optimizer_args={"lr": 1e-4}
    )
    return task

In [79]:
def trainingModel(maxEpochs):
    logger = pl.loggers.TensorBoardLogger(save_dir=qm9tut)
    callbacks = [
        spk.train.ModelCheckpoint(
            model_path = os.path.join(qm9tut, "best_inference_model"),
            save_top_k = 1,
            monitor = "val_loss"
        )
    ]
    trainer = pl.Trainer(
        callbacks = callbacks,
        logger = logger,
        default_root_dir = qm9tut,
        max_epochs = maxEpochs, 
    )
    trainer.fit(task, datamodule=qm9data)

## Creant el model:

In [80]:
import os
import schnetpack as spk
from schnetpack.datasets import QM9
import schnetpack.transform as trn

import torch
import torchmetrics
import pytorch_lightning as pl

qm9tut = './qm9tut'
if not os.path.exists('qm9tut'):
    os.makedirs(qm9tut)

In [81]:
# Import data from the QM9 dataset: 
dbPath = './qm9tut/qm9.db'
qm9data = ImportData(dbPath,100,1000,1000) 

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 10.57it/s]


In [82]:
# Settup model: 
nnpot = settingModel(5.0,30)

# Settup output:
output_U0 = settingOutput()

# Define task:
task = definingTasck()

In [83]:
# Training model:
trainingModel(4)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | NeuralNetworkPotential | 16.4 K
1 | outputs | ModuleList             | 0     
---------------------------------------------------
16.4 K    Trainable params
0         Non-trainable params
16.4 K    Total params
0.066     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=4` reached.


## Prediccio: 

In [84]:
import torch
import numpy as np
from ase import Atoms

best_model = torch.load(os.path.join(qm9tut, 'best_inference_model'))

We can use the test dataloader from the QM( data to obtain a batch of molecules and apply the model:

In [85]:
for batch in qm9data.test_dataloader():
    result = best_model(batch)
    print("Result dictionary:", result)
    break

Result dictionary: {'energy_U0': tensor([-10970.2609, -11372.6771, -10326.9094, -11373.6629, -10362.4389,
        -11474.5606, -11982.3176, -10868.4537, -11911.4700, -11545.2972,
        -11877.3117, -10568.0880, -12347.7429, -10531.7071,  -9407.4588,
        -11372.3639, -10533.3572,  -9924.9355, -11373.9016, -10567.0577,
        -12921.8328,  -9465.2620, -11948.0569, -10935.9655, -10969.8692,
        -10799.5144, -10337.8862, -10498.6623, -11577.4793,  -9901.7211,
        -10798.7374, -12485.5719, -12211.5524,  -9558.0435, -11910.8407,
        -11981.3731, -11947.0661, -12557.8185, -10833.4676, -12382.7335,
        -11372.0051,  -9900.8664, -13760.6083, -11476.1269, -12922.3523,
        -10842.0200, -10534.2791, -10533.5549, -11945.9532, -12348.1010,
        -10303.1538, -10567.5817, -12885.5084, -10971.1998,  -9534.0287,
        -12922.5249, -10968.9931,  -9901.1323, -10534.1485, -11510.1859,
        -10568.6562,  -9523.9487,  -8924.6351, -10532.4927, -11200.4019,
        -11510.168

If your data is not already in SchNetPack format, a convenient way is to use ASE atoms with the
provided `AtomsConverter`:

In [86]:
converter = spk.interfaces.AtomsConverter(neighbor_list=trn.ASENeighborList(cutoff=5.), dtype=torch.float32)

In [87]:
numbers = np.array([6, 1, 1, 1, 1])
positions = np.array([[-0.0126981359, 1.0858041578, 0.0080009958],
                      [0.002150416, -0.0060313176, 0.0019761204],
                      [1.0117308433, 1.4637511618, 0.0002765748],
                      [-0.540815069, 1.4475266138, -0.8766437152],
                      [-0.5238136345, 1.4379326443, 0.9063972942]])
atoms = Atoms(numbers=numbers, positions=positions)

In [88]:
inputs = converter(atoms)

print('Keys:', list(inputs.keys()))

pred = best_model(inputs)

print('Prediction:', pred[QM9.U0])

Keys: ['_n_atoms', '_atomic_numbers', '_positions', '_cell', '_pbc', '_idx', '_idx_i_local', '_idx_j_local', '_offsets', '_idx_m', '_idx_i', '_idx_j']
Prediction: tensor([-1106.2898], dtype=torch.float64, grad_fn=<AddBackward0>)


Alternatively, one can use the `SpkCalculator` as an interface to ASE. The calculator requires the path to a trained model and a neighborlist as input. In addition, the names and units of properties used in the model (e.g. the energy) should be provided. Precision and device can be set via the `dtype` and `device` keywords:

In [89]:
calculator = spk.interfaces.SpkCalculator(
    model_file=os.path.join(qm9tut, "best_inference_model"), # path to model
    neighbor_list=trn.ASENeighborList(cutoff=5.), # neighbor list
    energy_key=QM9.U0, # name of energy property in model
    energy_unit="eV", # units of energy property
    device="cpu", # device for computation
)
atoms.set_calculator(calculator)
print('Prediction:', atoms.get_total_energy())

INFO:schnetpack.interfaces.ase_interface:Loading model from ./qm9tut/best_inference_model


Prediction: -1106.2898198962212


The calculator automatically converts the prediction of the given unit to internal ASE units, which is `eV`
for the energy.
Using the calculator interface makes more sense if you have trained SchNet for a potential energy surface.
In the next tutorials, we will show how to learn potential energy surfaces and forces field as well as performing
molecular dynamics simulations with SchNetPack.